# 109: Swissmetro Nested Logit Mode Choice

In [ ]:
# TEST
import larch
import os
import pandas as pd
pd.set_option("display.max_columns", 999)
pd.set_option('expand_frame_repr', False)
pd.set_option('display.precision', 3)
larch._doctest_mode_ = True
import larch.numba as lx

In [ ]:
import pandas as pd
import larch.numba as lx

This example is a mode choice model built using the Swissmetro example dataset.
First we create the Dataset and Model objects. 

In [ ]:
raw_data = pd.read_csv(lx.example_file('swissmetro.csv.gz')).rename_axis(index='CASEID')
data = lx.Dataset.from_idco(raw_data, alts={1:'Train', 2:'SM', 3:'Car'})
m = lx.Model(data.query_cases("PURPOSE in (1,3) and CHOICE != 0"))

We can attach a title to the model. The title does not affect the calculations
as all; it is merely used in various output report styles.

In [ ]:
m.title = "swissmetro example 09 (nested logit)"

We need to identify the availability and choice variables.

In [ ]:
m.availability_co_vars = {
    1: "TRAIN_AV * (SP!=0)",
    2: "SM_AV",
    3: "CAR_AV * (SP!=0)",
}
m.choice_co_code = 'CHOICE'

The swissmetro dataset, as with all Biogeme data, is only in `co` format.

In [ ]:
from larch.roles import P,X
m.utility_co[1] = ( P.ASC_TRAIN
                  + P.B_TIME * X.TRAIN_TT
                  + P.B_COST * X("TRAIN_CO*(GA==0)") )
m.utility_co[2] = ( P.B_TIME * X.SM_TT
                  + P.B_COST * X("SM_CO*(GA==0)") )
m.utility_co[3] = ( P.ASC_CAR
                  + P.B_TIME * X.CAR_TT
                  + P.B_COST * X("CAR_CO") )

To create a new nest, we can use the `graph.new_node` command.
For this example, we want to nest together the Train and Car modes into a "existing" modes nest.
Those are modes 1 and 3, so we can use the `graph.new_node` command like this:

In [ ]:
m.graph.new_node(parameter="existing", children=[1,3], name='Existing')

In a Jupyter notebook, we can verify the nesting structure visually if we like.

In [ ]:
m.graph

Larch will find all the parameters in the model, but we'd like to output them in
a rational order.  We can use the ordering method to do this:

In [ ]:
m.ordering = [
    ("ASCs", 'ASC.*',),
    ("LOS", 'B_.*',),
    ("LogSums", 'existing'),
]

In [ ]:
# TEST
from pytest import approx
assert m.loglike() == approx(-6964.662979192185)

We can estimate the models and check the results match up with those given by Biogeme:

In [ ]:
m.set_cap(15)
m.maximize_loglike()

In [ ]:
# TEST
r = _
assert r.loglike == approx(-5236.900013608126)
assert r.logloss == approx(0.7737736426725954)
assert r.x.to_dict() == approx({
    'ASC_CAR': -0.16715395674702274,
    'ASC_TRAIN': -0.5119486070573562,
    'B_COST': -0.008566724901779925,
    'B_TIME': -0.00898662581404941,
    'existing': 0.4868411287532135,
})

In [ ]:
m.calculate_parameter_covariance()
m.parameter_summary()

The `pfo` (parameter frame, ordered) has other useful information,
including things like the robust standard errors and t statistics.

In [ ]:
print(m.pfo()[['value','std_err','t_stat','robust_std_err','robust_t_stat']])

In [ ]:
# TEST
assert str(m.pfo()[['value','std_err','t_stat','robust_std_err','robust_t_stat']]) == '''
                    value    std_err  t_stat  robust_std_err  robust_t_stat
Category Parameter                                                         
ASCs     ASC_CAR   -0.167  3.714e-02  -4.501       5.453e-02         -3.065
         ASC_TRAIN -0.512  4.518e-02 -11.331       7.911e-02         -6.471
LOS      B_COST    -0.009  4.627e-04 -18.513       6.004e-04        -14.269
         B_TIME    -0.009  5.699e-04 -15.769       1.071e-03         -8.390
LogSums  existing   0.487  2.790e-02 -18.394       3.892e-02        -13.186
'''[1:-1]